Team:
 Javidan Hajiyev,Aziz Zeynalli

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

heart_data = pd.read_csv('heart.csv')
features, target = heart_data.drop('target', axis=1), heart_data['target']

In [ ]:
def normalize(data, mean_vals=None, std_vals=None):
    if mean_vals is None:
        mean_vals = np.mean(data, axis=0)
    if std_vals is None:
        std_vals = np.std(data, axis=0)
    normalized_data = (data - mean_vals) / (std_vals + 1e-8)
    return normalized_data, mean_vals, std_vals

In [ ]:
def split_data(input_data, labels, test_fraction=0.25, seed_value=None):
    if seed_value is not None:
        np.random.seed(seed_value)

    total_samples = len(input_data)
    test_sample_count = int(test_fraction * total_samples)

    shuffled_indices = np.arange(total_samples)
    np.random.shuffle(shuffled_indices)

    if isinstance(input_data, pd.DataFrame):
        input_data = input_data.reset_index(drop=True).to_numpy()
    if isinstance(labels, pd.DataFrame):
        labels = labels.reset_index(drop=True).to_numpy()

    train_data = input_data[shuffled_indices[test_sample_count:]]
    train_labels = labels[shuffled_indices[test_sample_count:]]
    test_data = input_data[shuffled_indices[:test_sample_count]]
    test_labels = labels[shuffled_indices[:test_sample_count]]

    return train_data, test_data, train_labels, test_labels

In [ ]:
train_data, test_data, train_labels, test_labels = split_data(features, target, seed_value=42)
train_labels, test_labels = train_labels.to_numpy(), test_labels.to_numpy()

train_labels = train_labels.reshape(-1, 1)
test_labels = test_labels.reshape(-1, 1)

train_data_norm, mean_values_train, std_values_train = normalize(train_data)
test_data_norm = normalize(test_data, mean_values_train, std_values_train)[0]

In [ ]:
class MultiLayerPerceptron:
    def __init__(self, input_dimension, hidden_layer_units, output_dimension, learning_rate):
        self.input_dimension = input_dimension
        self.hidden_layer_units = hidden_layer_units
        self.output_dimension = output_dimension
        self.learning_rate = learning_rate

        self.weights_input_to_hidden = np.random.randn(self.input_dimension, self.hidden_layer_units)
        self.weights_hidden_to_output = np.random.randn(self.hidden_layer_units, self.output_dimension)

    def logistic_activation(self, input_data):
        return 1 / (1 + np.exp(-input_data))

    def logistic_derivative(self, output):
        return output * (1 - output)

    def forward_propagation(self, input_data):
        self.input_to_hidden = np.dot(input_data, self.weights_input_to_hidden)
        self.hidden_output = self.logistic_activation(self.input_to_hidden)
        self.hidden_to_output = np.dot(self.hidden_output, self.weights_hidden_to_output)
        self.final_output = self.logistic_activation(self.hidden_to_output)

    def backward_propagation(self, input_data, actual_labels):
        self.output_error = actual_labels - self.final_output
        self.output_delta = self.output_error * self.logistic_derivative(self.final_output)

        self.hidden_error = self.output_delta.dot(self.weights_hidden_to_output.T)
        self.hidden_delta = self.hidden_error * self.logistic_derivative(self.hidden_output)

        self.weights_hidden_to_output += self.hidden_output.T.dot(self.output_delta) * self.learning_rate
        self.weights_input_to_hidden += input_data.T.dot(self.hidden_delta) * self.learning_rate

    def train(self, train_data, train_labels, test_data, test_labels, num_epochs, batch_size, tolerance=1e-5):
        self.error_history = []
        self.epoch_history = []

        for epoch in range(num_epochs):
            for i in range(0, len(train_data), batch_size):
                batch_data = train_data[i:i+batch_size]
                batch_labels = train_labels[i:i+batch_size]
                self.forward_propagation(batch_data)
                self.backward_propagation(batch_data, batch_labels)

            self.forward_propagation(test_data)
            error = np.mean(np.abs(test_labels - self.final_output))
            self.error_history.append(error)
            self.epoch_history.append(epoch)

            if epoch > 1 and np.abs(self.error_history[-1] - self.error_history[-2]) < tolerance:
                break

    def make_prediction(self, input_data):
        self.forward_propagation(input_data)
        return np.round(self.final_output)

    def calculate_confusion_matrix(self, input_data, actual_labels):
        predicted_labels = self.make_prediction(input_data)
        true_positives = np.sum((predicted_labels == 1) & (actual_labels == 1))
        false_negatives = np.sum((predicted_labels == 0) & (actual_labels == 1))
        false_positives = np.sum((predicted_labels == 1) & (actual_labels == 0))
        true_negatives = np.sum((predicted_labels == 0) & (actual_labels == 0))
        return true_positives, false_negatives, false_positives, true_negatives

    def calculate_precision(self, input_data, actual_labels):
        true_positive, _, false_positive, _ = self.calculate_confusion_matrix(input_data, actual_labels)
        return true_positive / (true_positive + false_positive) if true_positive + false_positive != 0 else 0

    def calculate_accuracy(self, input_data, actual_labels):
        true_positive, true_negative, false_positive, false_negative = self.calculate_confusion_matrix(input_data, actual_labels)
        return (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative) if true_positive + true_negative + false_positive + false_negative != 0 else 0

    def calculate_sensitivity(self, input_data, actual_labels):
        true_positive, _, _, false_negative = self.calculate_confusion_matrix(input_data, actual_labels)
        return true_positive / (true_positive + false_negative) if true_positive + false_negative != 0 else 0

    def calculate_specificity(self, input_data, actual_labels):
        _, true_negative, false_positive, _ = self.calculate_confusion_matrix(input_data, actual_labels)
        return true_negative / (true_negative + false_positive) if true_negative + false_positive != 0 else 0

In [ ]:
neural_network = MultiLayerPerceptron(input_dimension=train_data_norm.shape[1], hidden_layer_units=5, output_dimension=1, learning_rate=0.01)
neural_network.train(train_data=train_data_norm, train_labels=train_labels, test_data=test_data_norm, test_labels=test_labels, num_epochs=1000, batch_size=4)

In [ ]:
precision = neural_network.calculate_precision(test_data_norm, test_labels)
accuracy = neural_network.calculate_accuracy(test_data_norm, test_labels)
sensitivity = neural_network.calculate_sensitivity(test_data_norm, test_labels)
specificity = neural_network.calculate_specificity(test_data_norm, test_labels)

print(f"precision is {precision*100} %")
print(f"accuracy is {accuracy*100} %")
print(f"sensitivity is {sensitivity*100} %")
print(f"specificity is {specificity*100} %")

precision is 85.36585365853658 %
accuracy is 53.333333333333336 %
sensitivity is 54.6875 %
specificity is 45.45454545454545 %


## DECISION TREE

In [ ]:
def calculate_entropy(actual_labels):
    histogram = np.bincount(actual_labels)
    probabilities = histogram / len(actual_labels)
    return -np.sum([probability * np.log2(probability) for probability in probabilities if probability > 0])

In [ ]:
def calculate_confusion_matrix(actual_labels, predicted_labels):
    true_positive = sum((actual_labels == 1) & (predicted_labels == 1))
    true_negative = sum((actual_labels == 0) & (predicted_labels == 0))
    false_positive = sum((actual_labels == 0) & (predicted_labels == 1))
    false_negative = sum((actual_labels == 1) & (predicted_labels == 0))
    return true_positive, true_negative, false_positive, false_negative

def calculate_precision(actual_labels, predicted_labels):
    true_positive, _, false_positive, _ = calculate_confusion_matrix(actual_labels, predicted_labels)
    return true_positive / (true_positive + false_positive) if true_positive + false_positive != 0 else 0

def calculate_accuracy(actual_labels, predicted_labels):
    accuracy = np.sum(actual_labels == predicted_labels) / len(actual_labels)
    return accuracy

def calculate_sensitivity(actual_labels, predicted_labels):
    true_positive, _, _, false_negative = calculate_confusion_matrix(actual_labels, predicted_labels)
    return true_positive / (true_positive + false_negative) if true_positive + false_negative != 0 else 0

def calculate_specificity(actual_labels, predicted_labels):
    _, true_negative, false_positive, _ = calculate_confusion_matrix(actual_labels, predicted_labels)
    return true_negative / (true_negative + false_positive) if true_negative + false_positive != 0 else 0

In [ ]:
class TreeNode:
    def __init__(self, predicted_value, index=0, cutoff=0, left_branch=None, right_branch=None):
        self.predicted_value = predicted_value
        self.index = index
        self.cutoff = cutoff
        self.left_branch = left_branch
        self.right_branch = right_branch

    def is_leaf_node(self):
        return self.left_branch is None and self.right_branch is None

In [ ]:
from collections import Counter
import numpy as np
import pandas as pd

class DecisionTree:
    def __init__(self, df, target_column, test_size=0.25, min_samples=2, max_tree_depth=50, num_features=None):
        self.df = df
        self.target_column = target_column
        self.test_size = test_size
        self.min_samples = min_samples
        self.max_tree_depth = max_tree_depth
        self.num_features = num_features
        self.root_node = None

        self.features_train = None
        self.features_test = None
        self.labels_train = None
        self.labels_test = None

        self._preprocess_data()

    def _preprocess_data(self):
        x = self.df.drop(self.target_column, axis=1)
        y = self.df[self.target_column]

        indices = np.arange(x.shape[0])
        np.random.shuffle(indices)
        split_idx = int(x.shape[0] * (1 - self.test_size))

        self.features_train, self.features_test = x.iloc[indices[:split_idx], :].values, x.iloc[indices[split_idx:], :].values
        self.labels_train, self.labels_test = y.iloc[indices[:split_idx]].values, y.iloc[indices[split_idx:]].values

    def fit(self):
        self.num_features = self.features_train.shape[1] if not self.num_features else min(self.num_features, self.features_train.shape[1])
        self.root_node = self._grow_tree(self.features_train, self.labels_train)

    def predict(self):
        return np.array([self._traverse_tree(feature, self.root_node) for feature in self.features_test])

    def _grow_tree(self, features, labels, depth=0):
        num_samples, num_features = features.shape
        num_labels = len(np.unique(labels))

        if (depth >= self.max_tree_depth
                or num_labels == 1
                or num_samples < self.min_samples):
            leaf_value = self._most_common_label(labels)
            return TreeNode(predicted_value=leaf_value)

        feature_indices = np.random.choice(num_features, self.num_features, replace=False)

        best_feature, best_cutoff = self._best_criteria(features, labels, feature_indices)
        best_cutoff = float(best_cutoff)

        left_indices, right_indices = self._split(features[:, best_feature], best_cutoff)
        left_branch = self._grow_tree(features[left_indices, :], labels[left_indices], depth+1)
        right_branch = self._grow_tree(features[right_indices, :], labels[right_indices], depth+1)
        return TreeNode(predicted_value=self._most_common_label(labels), index=best_feature, cutoff=best_cutoff, left_branch=left_branch, right_branch=right_branch)

    def _best_criteria(self, features, labels, feature_indices):
        best_gain = -1
        split_index, split_cutoff = None, None
        for feature_index in feature_indices:
            feature_column = features[:, feature_index]
            cutoffs = np.unique(feature_column)
            for cutoff in cutoffs:
                gain = self._information_gain(labels, feature_column, cutoff)

                if gain > best_gain:
                    best_gain = gain
                    split_index = feature_index
                    split_cutoff = cutoff

        return split_index, split_cutoff

    def _information_gain(self, labels, feature_column, split_cutoff):
        parent_entropy = calculate_entropy(labels)

        left_indices, right_indices = self._split(feature_column, split_cutoff)

        if len(left_indices) == 0 or len(right_indices) == 0:
            return 0

        total = len(labels)
        num_left, num_right = len(left_indices), len(right_indices)
        entropy_left, entropy_right = calculate_entropy(labels[left_indices]), calculate_entropy(labels[right_indices])
        child_entropy = (num_left / total) * entropy_left + (num_right / total) * entropy_right

        info_gain = parent_entropy - child_entropy
        return info_gain

    def _split(self, feature_column, split_cutoff):
        feature_column = np.array(feature_column)
        left_indices = np.argwhere(feature_column <= split_cutoff).flatten()
        right_indices = np.argwhere(feature_column > split_cutoff).flatten()
        return left_indices, right_indices

    def _traverse_tree(self, feature, node):
        if node.is_leaf_node():
            return node.predicted_value

        if feature[node.index] <= node.cutoff:
            return self._traverse_tree(feature, node.left_branch)
        return self._traverse_tree(feature, node.right_branch)

    def _most_common_label(self, labels):
        counter = Counter(labels)
        most_common = counter.most_common(1)[0][0]
        return most_common

In [ ]:
tree = DecisionTree(df=heart_data, target_column='target', max_tree_depth=3)

tree.fit()

predicted_labels = tree.predict()

In [ ]:
accuracy_score = calculate_accuracy(tree.labels_test, predicted_labels)
precision_score = calculate_precision(tree.labels_test, predicted_labels)
sensitivity_score = calculate_sensitivity(tree.labels_test, predicted_labels)
specificity_score = calculate_specificity(tree.labels_test, predicted_labels)

print(f"precision is {precision_score*100} %")
print(f"accuracy is {accuracy_score*100} %")
print(f"sensitivity is {sensitivity_score*100} %")
print(f"specificity is {specificity_score*100} %")

precision is 82.97872340425532 %
accuracy is 86.8421052631579 %
sensitivity is 95.1219512195122 %
specificity is 77.14285714285715 %
